In [112]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TEMP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [113]:
df1 = pd.read_csv('Dataset_clean(penelitian).csv')
df2 = pd.read_csv('Dataset_clean(aplikasi).csv')

train_data = pd.concat([df1, df2])

In [114]:
train_data = train_data[['full_text', 'emotion']]
train_data

train_data = train_data.dropna()
#hitung baris yang memiliki kata Skipped due to content policy
skipped_rows = train_data[train_data['full_text'] == 'Skipped due to content policy']
print("Number of rows with 'Skipped due to content policy':", len(skipped_rows))


Number of rows with 'Skipped due to content policy': 0


In [115]:
def tokenization_sentences(tweet):
    if isinstance(tweet, list):
        tokenized_tweet = []
        for word in tweet:
            tokens = word_tokenize(word)
            tokenized_tweet.extend(tokens)
        return tokenized_tweet
    else:
        return word_tokenize(tweet)

In [116]:
#terapkan fungsi tokenization_sentences pada kolom 'full_text'
train_data['full_text'] = train_data['full_text'].apply(tokenization_sentences)

In [117]:
train_data

,full_text,emotion
0,"[pengabdian, terpuji, meskipun, terlihat, biasa]",Neutral
1,"[baru, mengirim, foto]",Neutral
2,"[apa, kabar, dunia, twitter, masihkah, selucu,...",Neutral
3,"[yah, gimana, sih, yaudah, susul, sebelumnya, ...",Neutral
4,"[ka, dorg, suro, beli, medium, set]",Neutral
...,...,...
28614,"[cod, beli, euphoria, wonderland, la, foret, f...",Joy
28615,"[ลองด, cod, flash, sale, ก, โล, แมคคายกกระสอบ,...",Joy
28616,"[wal, cod, an, shopee, kawa, pakai, qris, lah,...",Neutral
28617,"[temukan, bisa, cod, light, stick, treasure, b...",Neutral


In [118]:
jumlah_netral = len(train_data[train_data['emotion'] == "Neutral"])
jumlah_marah = len(train_data[train_data['emotion'] == "Anger"])
jumlah_senang = len(train_data[train_data['emotion'] == "Joy"])
jumlah_cinta = len(train_data[train_data['emotion'] == "Love"])
jumlah_sedih = len(train_data[train_data['emotion'] == "Sad"])
jumlah_takut = len(train_data[train_data['emotion'] == "Fear"])

In [119]:
print("Jumlah Netral: ", jumlah_netral)
print("Jumlah Marah: ", jumlah_marah)
print("Jumlah Senang: ", jumlah_senang)
print("Jumlah Cinta: ", jumlah_cinta)
print("Jumlah Sedih: ", jumlah_sedih)
print("Jumlah Takut: ", jumlah_takut)

Jumlah Netral:  12981
Jumlah Marah:  11069
Jumlah Senang:  12307
Jumlah Cinta:  5389
Jumlah Sedih:  7272
Jumlah Takut:  7996


In [120]:
import pandas as pd
import random

# Set the random seed for reproducibility
random.seed(42)

# Filter out the rows with the specified emotions
filtered_data = train_data[~train_data['emotion'].isin(["Skipped due to content policy"])]

# Define the total maximum count and the range
total_max_count = 56714
range_diff = 2031

# Define the order of emotions
emotion_order = ["Neutral", "Anger", "Joy", "Love", "Sad", "Fear"]

# Initialize a list to store the balanced data
balanced_data = []

# Calculate the number of labels
num_labels = len(emotion_order)

# Define the base count for each label to ensure we stay within total_max_count
base_count = total_max_count // num_labels

# Calculate the minimum and maximum count for each label
min_count = base_count - range_diff // 2
max_count = base_count + range_diff // 2

# Calculate random target counts ensuring they are in descending order
target_counts = []
remaining_count = total_max_count

for i in range(num_labels):
    if i == num_labels - 1:
        # Assign the remaining count to the last emotion
        target_counts.append(remaining_count)
    else:
        max_count = min(max_count, remaining_count - (num_labels - i - 1) * min_count)
        target_count = random.randint(min_count, max_count)
        target_counts.append(target_count)
        remaining_count -= target_count

# Ensure target_counts is sorted in descending order
target_counts.sort(reverse=True)

# Iterate over each emotion in the defined order and balance the dataset
for emotion, target_count in zip(emotion_order, target_counts):
    emotion_data = filtered_data[filtered_data['emotion'] == emotion]
    current_count = len(emotion_data)
    
    if current_count > target_count:
        # If current count is greater than target, sample down
        sampled_data = emotion_data.sample(target_count, random_state=42)
    else:
        # If current count is less than target, duplicate up
        additional_data = emotion_data.sample(target_count - current_count, replace=True, random_state=42)
        sampled_data = pd.concat([emotion_data, additional_data])
    
    balanced_data.append(sampled_data)

# Concatenate all the balanced data
final_balanced_data = pd.concat(balanced_data).reset_index(drop=True)

# Adjust the total number of entries to be exactly 56,714
current_total = len(final_balanced_data)
if current_total > total_max_count:
    final_balanced_data = final_balanced_data.sample(total_max_count, random_state=42)
elif current_total < total_max_count:
    additional_data = final_balanced_data.sample(total_max_count - current_total, replace=True, random_state=42)
    final_balanced_data = pd.concat([final_balanced_data, additional_data])


In [121]:
# Display the new counts
print(final_balanced_data['emotion'].value_counts())

emotion
Neutral    10860
Anger       9955
Joy         9746
Love        9000
Sad         8665
Fear        8488
Name: count, dtype: int64


In [122]:
final_balanced_data

,full_text,emotion
0,"[wakil, ketua, komisi, i, dpr, utut, adianto, ...",Neutral
1,"[kemnaker, rumah, sakit, bersinergi, terapkan,...",Neutral
2,"[very, interesting, comment, sheds, some, ligh...",Neutral
3,"[distribusi, logistik, pemilu, tahap, pertama,...",Neutral
4,"[well, it, is, geopolitics, you, see, russia, ...",Neutral
...,...,...
56709,"[mungkin, uang, hasil, jualan, pulau, amp, uan...",Fear
56710,"[joe, biden, is, caught, in, a, trap, caused, ...",Fear
56711,"[by, the, way, iya, benar, ngeri, sih, olx, se...",Fear
56712,"[facebook, dan, teman, teman, mengalami, gangg...",Fear


In [123]:
# Save the balanced dataset
final_balanced_data.to_csv('balanced_train_data.csv', index=False)